## Actividad 3: Traductor español - ingles 
Diego Arturo Padilla Domínguez - A01552594

Keyuan Zhao                    - A01366831

Carolina Herrera Martínez      - A01411547

Cutberto Arizabalo Nava        - A01411431

Jose Pablo Cobos Austria       - A01274631

### Recursos utilizados para el proyecto

https://www.manythings.org/anki/

https://tatoeba.org/en/

https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

#### Limpieza e obtencion de datos

##### Desde scratch

In [1]:
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import os
os.chdir("/home/josecobos/Documentos/Escuela/IA_Parte2/Modulo_3")
a = os.getcwd()
print(a)
if torch.cuda.is_available:
    device = torch.device("cuda")
    print("Cuda device selected")
else:
    device = torch.device("cpu")
    print("Cpu device selected")

/home/josecobos/Documentos/Escuela/IA_Parte2/Modulo_3
Cuda device selected


In [2]:
from lang_processing_good import *

Working Lang Processing Module


In [3]:
lines = open('/home/josecobos/Documentos/Escuela/IA_Parte2/Modulo_3/data/t.txt', encoding='utf-8').read().strip().split('\n')

In [4]:
input_lang, output_lang, pairs = prepareData('eng','spa', True)

Reading lines...
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Counting words...
Counted words:
spa 4345
eng 2803


In [5]:
MAX_LENGTH = 10

In [6]:
print(len(pairs))

10599


In [7]:
# rnd_idx = 
print(pairs[11:20])

[['je suis mouille .', 'i m wet .'], ['je suis mouillee .', 'i m wet .'], ['il est mouille .', 'he s wet .'], ['je suis gras .', 'i am fat .'], ['je suis revenu .', 'i m back .'], ['me revoila .', 'i m back .'], ['je suis chauve .', 'i m bald .'], ['je suis occupe .', 'i m busy .'], ['je suis occupee .', 'i m busy .']]


In [8]:
input_lang.word2index

{'j': 2,
 'ai': 3,
 'ans': 4,
 '.': 5,
 'je': 6,
 'vais': 7,
 'bien': 8,
 'ca': 9,
 'va': 10,
 'suis': 11,
 'gras': 12,
 'gros': 13,
 'en': 14,
 'forme': 15,
 'touche': 16,
 '!': 17,
 'touchee': 18,
 'malade': 19,
 'triste': 20,
 'timide': 21,
 'mouille': 22,
 'mouillee': 23,
 'il': 24,
 'est': 25,
 'revenu': 26,
 'me': 27,
 'revoila': 28,
 'chauve': 29,
 'occupe': 30,
 'occupee': 31,
 'calme': 32,
 'froid': 33,
 'fini': 34,
 'tout': 35,
 'libre': 36,
 'disponible': 37,
 'repu': 38,
 'rassasie': 39,
 'content': 40,
 'chez': 41,
 'moi': 42,
 'retard': 43,
 'paresseux': 44,
 'faineant': 45,
 'paresseuse': 46,
 'faineante': 47,
 'porte': 48,
 'securite': 49,
 'certain': 50,
 'sur': 51,
 'sure': 52,
 'grande': 53,
 'mince': 54,
 'ordonne': 55,
 'ordonnee': 56,
 'laid': 57,
 'laide': 58,
 'faible': 59,
 'vieux': 60,
 'dj': 61,
 'bon': 62,
 'riche': 63,
 'ici': 64,
 'flic': 65,
 'un': 66,
 'homme': 67,
 'seule': 68,
 'seul': 69,
 'arme': 70,
 'armee': 71,
 'reveille': 72,
 'aveugle': 73,
 'f

### Creacion del Modelo GRU

In [9]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, x, hidden):
        embedded = self.embedding(x).view(1, 1, -1)
        output, hidden = self.gru(embedded, hidden)
        return output, hidden

    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [10]:
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super().__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x, hidden):
        output = self.embedding(x).view(1, 1, -1)
#         output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        # print(output.shape)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [11]:
class AttnDecoder(nn.Module):
    def __init__(self, hidden_size, output_size, max_length=MAX_LENGTH):
        super().__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
#         self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
#         self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, x, hidden, encoder_outputs):
        embedded = self.embedding(x).view(1, 1, -1)
#         embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

### Entrenamiento del modelo

In [12]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [13]:
teacher_forcing_ratio = 0.5


def train_once(input_tensor, target_tensor, encoder, decoder,
          encoder_optimizer, decoder_optimizer, criterion, 
          max_length=MAX_LENGTH, attention=True):
    encoder_hidden = encoder.init_hidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):

            if attention:
              decoder_output, decoder_hidden, decoder_attention = decoder(
                  decoder_input, decoder_hidden, encoder_outputs)
            else:
              decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)


            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):

          if attention:
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
          else:
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)


          topv, topi = decoder_output.topk(1)
          decoder_input = topi.squeeze().detach()  # detach from history as input

          loss += criterion(decoder_output, target_tensor[di])
          if decoder_input.item() == EOS_token:
              break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [14]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [15]:
random.choice(pairs)

['vous etes en danger .', 'you re in danger .']

### Imprimiendo resultados

In [16]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [17]:
def train(encoder, decoder, n_iters, print_every=1000, 
               plot_every=100, learning_rate=0.01, attention=True):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train_once(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, 
                     criterion, attention=attention)
        
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

#### Evaluacion del Modelo

In [18]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH, attention=True):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.init_hidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
          if attention:
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
          else:
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)


          if topi.item() == EOS_token:
              decoded_words.append('<EOS>')
              break
          else:
              decoded_words.append(output_lang.index2word[topi.item()])

          decoder_input = topi.squeeze().detach()

        if attention:
          return decoded_words, decoder_attentions[:di + 1]
        else:
          return decoded_words

In [19]:
def evaluateRandomly(encoder, decoder, n=10, attention=True):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        if attention:
          output_words, attentions = evaluate(encoder, decoder, pair[0], attention=attention)
        else:
          output_words = evaluate(encoder, decoder, pair[0], attention=False)

        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [20]:
hidden_size = 256
encoder1 = Encoder(input_lang.n_words, hidden_size).to(device)
decoder1 = AttnDecoder(hidden_size, output_lang.n_words).to(device)
#decoder1 = Decoder(hidden_size, output_lang.n_words).to(device)
train(encoder1, decoder1, 50000, print_every=5000, attention=True)

1m 0s (- 9m 4s) (5000 10%) 2.8430
1m 58s (- 7m 54s) (10000 20%) 2.2646
2m 57s (- 6m 53s) (15000 30%) 1.9594
3m 55s (- 5m 53s) (20000 40%) 1.7313
4m 53s (- 4m 53s) (25000 50%) 1.4908
5m 51s (- 3m 54s) (30000 60%) 1.3472
6m 51s (- 2m 56s) (35000 70%) 1.2130
7m 55s (- 1m 58s) (40000 80%) 1.0825
8m 57s (- 0m 59s) (45000 90%) 0.9938
10m 1s (- 0m 0s) (50000 100%) 0.8838


In [21]:
evaluateRandomly(encoder1, decoder1, attention=True)

> il est membre du conseil paroissial .
= he is a member of the parish committee .
< he is the one of the result . <EOS>

> je suis souvent en difficulte .
= i m often in trouble .
< i m in trouble trouble . <EOS>

> je suis egalement occupee .
= i m busy too .
< i m busy too . <EOS>

> j attends toujours pour ma commande .
= i m still waiting for my order .
< i m still waiting for my car . <EOS>

> vous avez l air tres bien .
= you re looking very well .
< you re doing very good . <EOS>

> je ne suis pas joueur .
= i m not a gambler .
< i m not a . <EOS>

> vous etes endormi .
= you re sleepy .
< you re sleepy . <EOS>

> vous n etes pas differente .
= you re no different .
< you re no different . <EOS>

> je suis prete si tu l es .
= i m ready if you are .
< i m ready if you are . <EOS>

> vous n etes pas different .
= you re no different .
< you re no different . <EOS>

